In [24]:
import numpy as np
import pandas as pd
import seaborn as sns
from oneibl import one
from export_funs import trialinfo_to_df
%matplotlib inline
one = one.ONE(offline=True)

SMALL_SIZE = 16
MEDIUM_SIZE = 20
BIGGER_SIZE = 24

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

session_id = '259927fd-7563-4b03-bc5d-17b4d0fa7a55'
regdata = np.load('./fits/ZM_2240/2020-01-22_session_2020-07-13_probe0_pyfit_sklearn.p', allow_pickle=True)
trialsdf = trialinfo_to_df(session_id, wheel=False)
fits = regdata['fitobj']
weights = regdata['weights']
spk_times = one.load(session_id, dataset_types=['spikes.times'])[0]
spk_clu = one.load(session_id, dataset_types=['spikes.clusters'])[0]

In [25]:
import ipywidgets as iw
from brainbox.plot import peri_event_time_histogram
cells = weights['stimonL'].index.to_list()
kernels = list(weights.keys())
scores = fits.score()

@iw.interact
def plot_fits(cell=cells, kernel=kernels):
    if kernel[:-1] == 'stimon':
        side = 'contrastLeft' if kernel[-1] == 'L' else 'contrastRight'
        times = trialsdf[np.isfinite(trialsdf[side])].stimOn_times.values
    elif kernel[-4:] == 'rect':
        fdbk = 1 if kernel == 'correct' else -1
        times = trialsdf[trialsdf['feedbackType'] == fdbk].feedback_times

    if kernel in ['stimonL', 'stimonR', 'correct', 'incorrect']:
        fig, ax = plt.subplots(2, 1, figsize=(8, 12))
        ax0 = ax[0]
        ax1 = ax[1]
        peri_event_time_histogram(spk_times, spk_clu, times, cell, t_before=0., t_after=0.6, bin_size=0.02, ax=ax[0],
                                  error_bars='sem')
    else:
        fig, ax1 = plt.subplots(1, 1, figsize=(6, 6)) 
    colors = sns.cubehelix_palette(20, start=2, rot=0, dark=0, light=.8)
    ax1.plot(weights[kernel].loc[cell])
    plt.title(f'D^2 score of {scores[cell]}')
    plt.tight_layout()

interactive(children=(Dropdown(description='cell', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 16, 17, …

In [23]:
fits.variances

AttributeError: 'NeuralGLM' object has no attribute 'variances'